# Cart Rotation Analysis

This notebook is to analyze the cart rotation of M2 with the configuration files:

1. **ts_mtm2/config/sysconfig/Configurable_File_Description_20180831T092556_surrogate_handling.csv**.
2. **ts_mtm2/config/sysconfig/Configurable_File_Description_20180831T092423_surrogate_optical.csv**

The difference can follow [DM-36986](https://jira.lsstcorp.org/browse/DM-36986).
In the test, we use the surrogate instead of the actual mirror.

The target is to understand:

1. The look-up table (LUT) correction.
2. The force balance system is working or not.
3. The forward modeling in hardpoint correction.

It is noted that the correctness of force balance system would need a more detailed analysis in the future.

## Note

For the normal operation range, the cart rotation is in the clockwise direction (A1 goes up: +Y, A4 goes down).
When the cart is flat, (M2 face down), the telescope mount assembly (TMA) elevation angle is 90 degree and the zenith angle is 0 degree.
When the cart rotates to straight (clockwise), the TMA elevation angle is 0 degree and the zenith angle is 90 degree.
This rotation is between "A2+A3" and "A5+A6".
Because of the force direction of tangent link, A1 and A4 will not take the mirror's weight in the rotation process.

When the mirror is face down, the TMA elevation angle should be 90 degree and the M2 processed inclinometer also shows ~90 degree.

## Summary

1. For the TMA elevation angle between 0 and 90 degrees, the coordination system of M2 processed inclinometer angle (used in the LUT calculation) should be the same as TMA.
2. The axial actuator 20 (0-based) does not behave the same as others.

## Import Modules

This notebook needs to setup the **ts_m2com** and **ts_aos_utilsts** under the **notebooks/.user_setups**, which depends on the **ts_tcpip**.
You also need to have **ts_mtm2** under the **WORK/** directory to read the confiugration files to do the analysis.

In [ ]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from astropy.time import Time

from lsst.ts.m2com import MockControlClosedLoop

from lsst.ts.aos.utils import DiagnosticsM2

## Declaration of User-defined Functions

In [ ]:
async def show_all_data(
    diagnostics_m2: DiagnosticsM2,
    control_closed_loop: MockControlClosedLoop,
    time_start: Time,
    time_end: Time,
    thresholds: list[float] | None = None,
    hardpoints: list[int] | None = None,
) -> None:
    """
    Show all the data.

    Parameters
    ----------
    diagnostics_m2 : `lsst.ts.aos.utils.DiagnosticsM2`
        M2 diagnostics instance.
    control_closed_loop : `lsst.ts.m2.com.MockControlClosedLoop`
        Mock control closed loop instance.
    time_start : `astropy.time.core.Time`
        Start time.
    time_end : `astropy.time.core.Time`
        End time.
    thresholds : `list` or None, optional
        Force thresholds of the axial and tangent actuators. (the default is None)
    hardpoints : `list` or None, optional
        Ordered 0-based six hardpoints. The first three are the axial
        actuators and the latters are the tangent links. If not None, they
        will be labeled on the drawing. (the default is None)
    """

    # Get the x, y position of actuators
    xy_actuators = diagnostics_m2.get_xy_actuators(control_closed_loop)

    # Query data
    data_zenith_angle, time_operation_angle = await diagnostics_m2.get_data_zenith_angle(
        time_start, time_end, realign_time=False
    )

    data_step_axial, time_operation_axial = await diagnostics_m2.get_data_step_axial(time_start, time_end)
    data_step_tangent, time_operation_tangent = await diagnostics_m2.get_data_step_tangent(time_start, time_end)
    
    data_force, time_operation_force = await diagnostics_m2.get_data_net_force(
        time_start, time_end
    )
    data_moment, time_operation_moment = await diagnostics_m2.get_data_net_moment(
        time_start, time_end
    )

    data_force_balance, time_operation_force_balance = await diagnostics_m2.get_data_force_balance(
        time_start, time_end
    )

    data_collected_axial, data_collected_tangent = await diagnostics_m2.get_data_force(
        time_start, time_end
    )
    data_temperature = await diagnostics_m2.get_data_temperature(time_start, time_end)
    
    # Calculate the theoretical LUT force
    data_collected_axial, data_collected_tangent = diagnostics_m2.set_lut_force_theoretical(
        control_closed_loop,
        data_collected_axial,
        data_collected_tangent,
        data_temperature,
        data_zenith_angle,
        time_operation_angle,
    )

    # Draw the diagrams
    diagnostics_m2.plot_processed_inclinometer(
        data_zenith_angle, time_operation_angle-time_operation_angle[0]
    )

    diagnostics_m2.plot_net_force(data_force, time_operation_force)
    diagnostics_m2.plot_net_moment(data_moment, time_operation_moment)    

    diagnostics_m2.plot_force_balance(data_force_balance, time_operation_force_balance)

    diagnostics_m2.draw_values(
        xy_actuators,
        data_step_axial[-1,:]-data_step_axial[0,:],
        data_step_tangent[-1,:]- data_step_tangent[0,:],
        title="Step Difference between Destination and Origin",
        hardpoints=hardpoints,
    )

    diagnostics_m2.draw_values(
        xy_actuators,
        data_collected_axial["measured"][-1]-data_collected_axial["measured"][0],
        data_collected_tangent["measured"][-1]-data_collected_tangent["measured"][0],
        title="Force Difference between Destination and Origin",
        hardpoints=hardpoints,
    )
    
    diagnostics_m2.plot_force_error(data_collected_axial, data_collected_tangent)

    diagnostics_m2.plot_lut_force_error_axial(data_collected_axial)
    diagnostics_m2.plot_lut_force_error_tangent(data_collected_tangent)
    
    # Print the information
    if thresholds is not None:
        diagnostics_m2.print_actuator_force_error_out_threshold(
            data_collected_axial, thresholds[0], "axial"
        )
        diagnostics_m2.print_actuator_force_error_out_threshold(
            data_collected_tangent, thresholds[1], "tangent"
        )

## Instantiate the MockControlClosedLoop

In [ ]:
config_path = Path.home() / "WORK" / "ts_mtm2" / "config"

filepath_surrogate = config_path / "parameter_files" / "luts" / "FinalHandlingLUTs"
filepath_mirror = config_path / "parameter_files" / "luts" / "FinalOpticalLUTs"

filepath_cell_geom = config_path / "cellGeom.json"

In [ ]:
control_closed_loop_surrogate = MockControlClosedLoop()
control_closed_loop_surrogate.load_file_lut(filepath_surrogate)
control_closed_loop_surrogate.load_file_cell_geometry(filepath_cell_geom)
control_closed_loop_surrogate.set_hardpoint_compensation()

control_closed_loop_mirror = MockControlClosedLoop()
control_closed_loop_mirror.load_file_lut(filepath_mirror)
control_closed_loop_mirror.load_file_cell_geometry(filepath_cell_geom)
control_closed_loop_mirror.set_hardpoint_compensation()

## Instantiate the DiagnosticsM2 Class

Notice that the UTC time is used when doing the query.

In [ ]:
diagnostics_m2 = DiagnosticsM2(is_summit=True)
hardpoints = [5, 15, 25, 73, 75, 77]

## Configurable_File_Description_20180831T092556_surrogate_handling.csv

### Rotate the Cart from 0 to 90 Degree in Clockwise Direction

The cart's controller shows 0 - 270 degree.
This is the operation range of TMA with elevation angle from 90 degree to 0 degree.

In [ ]:
time_start = Time("2023-05-17T15:38:00", scale="utc", format="isot")
time_end = Time("2023-05-17T16:18:00", scale="utc", format="isot")

await show_all_data(
    diagnostics_m2,
    control_closed_loop_surrogate,
    time_start, time_end,
    thresholds=[5.0, 10.0],
    hardpoints=hardpoints,
)

### Rotate the Cart from 0 to -90 Degree in Counter-Clockwise Direction

The cart's controller shows 0 - 90 degree.

In [ ]:
time_start = Time("2023-05-17T16:29:50", scale="utc", format="isot")
time_end = Time("2023-05-17T17:04:30", scale="utc", format="isot")

await show_all_data(
    diagnostics_m2,
    control_closed_loop_surrogate,
    time_start, time_end,
    thresholds=[5.0, 15.0],
    hardpoints=hardpoints,
)

## Configurable_File_Description_20180831T092423_surrogate_optical.csv

### Rotate the Cart from 0 to 90 Degree in Clockwise Direction

The cart's controller shows 0 - 270 degree.
This is the operation range of TMA with elevation angle from 90 degree to 0 degree.

In [ ]:
time_start = Time("2023-05-17T19:02:00", scale="utc", format="isot")
time_end = Time("2023-05-17T19:30:00", scale="utc", format="isot")

await show_all_data(
    diagnostics_m2,
    control_closed_loop_mirror,
    time_start, time_end,
    thresholds=[3.0, 20.0],
    hardpoints=hardpoints,
)

### Rotate the Cart from 0 to -90 Degree in Counter-Clockwise Direction

The cart's controller shows 0 - 90 degree.

In [ ]:
time_start = Time("2023-05-17T19:37:00", scale="utc", format="isot")
time_end = Time("2023-05-17T20:03:00", scale="utc", format="isot")

await show_all_data(
    diagnostics_m2,
    control_closed_loop_mirror,
    time_start, time_end,
    thresholds=[3.0, 30.0],
    hardpoints=hardpoints,
)

In [ ]:
plt.close("all")